In [1]:
import os.path as op
import os

import subprocess

import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
from scipy.stats import pearsonr, ttest_ind, f_oneway
from scipy.io import wavfile
import statsmodels.formula.api as smf

from navec import Navec

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib_venn import venn3
%matplotlib qt

font = {'font.family' : 'Arial',
        'font.size'   : 7,
        'pdf.fonttype': 42}

plt.rcParams.update(font)
# single col: 3.386
# double col: 7.087
# lines 0.25-1 pt
# font at least 7

### Reading data

In [2]:
# demographic and clinical features
demo = pd.read_csv(op.join('data', 'demo.csv'))

# behavioral data
pwi_wide = pd.read_csv(op.join('data', 'PWI_data_25pat.csv'))
pwi_lemma_freq = pd.read_csv(op.join('data', 'PWI_lemma_frequencies.csv'))
pwi_lemmas = pd.read_csv(op.join('data', 'PWI_lemmatization.csv'))#.drop_duplicates(subset = 'Response')

# neural data
cortical_loads = pd.read_csv(op.join('data', 'harvoxf_batch_descriptives_25pat.csv'))
tract_loads = pd.read_csv(op.join('data', 'tractotron_output', 'proportion.csv'))
tract_probs = pd.read_csv(op.join('data', 'tractotron_output', 'probability.csv'))

# reformatting PWI data to long format
pwi_data = pd.DataFrame({})

common_cols = ['Critical.segment', 'Target', 'Distractor', 'Condition']

for sub in demo['ID'].unique():
    
    # subsetting current participant's data
    subset = pwi_wide.loc[:,pwi_wide.columns.str.startswith(sub)]
    subset = subset.rename(lambda c: c.replace(sub, '').strip('.'), axis=1)
    
    # adding common columns
    subset_full = pd.concat([pwi_wide.loc[:,common_cols], subset], axis=1)
    subset_full['ID'] = sub
    
    # concatenating to the full dataframe
    pwi_data = pd.concat([pwi_data, subset_full])

pwi_data['ResponseType'] = pwi_data['ResponseType'].astype(str)

# subsetting neural predictors
cortical_rois = {'Inferior Frontal Gyrus, pars opercularis': 'POper',
                 'Inferior Frontal Gyrus, pars triangularis': 'PTri',
                 'Precentral Gyrus': 'PreCG'}

wm_rois = {'Superior_Londgitudinal_Fasciculus_I_Left': 'SLF_I',
           'Superior_Londgitudinal_Fasciculus_II_Left': 'SLF_II',
           'Superior_Londgitudinal_Fasciculus_III_Left': 'SLF_III'}

# subsetting cortical rois
neural_predictors = cortical_loads.loc[cortical_loads['roi_name'].isin(list(cortical_rois.keys())),
                                       ['ID', 'roi_name', 'numVox', 'numVoxNotZero']]
neural_predictors['load'] = neural_predictors['numVoxNotZero'] / neural_predictors['numVox']

# long to wide
neural_predictors = neural_predictors.pivot(index='ID', columns='roi_name', values='load')
neural_predictors = neural_predictors.rename(cortical_rois, axis=1)

# nans to zeros
neural_predictors = neural_predictors.fillna(0)

# subcortical rois
tract_loads['ID'] = tract_loads['filename'].apply(lambda x: 'cprin' + x.split('_')[0])

tract_loads_subset = tract_loads.loc[:,['ID'] + list(wm_rois.keys())]
tract_loads_subset = tract_loads_subset.rename(wm_rois, axis=1).set_index('ID')

neural_predictors = pd.concat([neural_predictors, tract_loads_subset], axis=1).reset_index()

### Fetching stimulus and response features

* target-distractor dissimilarity
* sequential response similarity
* response frequency

All similarities are Pearson correlations between embeddings extracted from a pre-trained GloVe model for Russian (https://github.com/natasha/navec)

In [3]:
# reading navec model
navec = Navec.load(op.join('utils', 'navec_hudlit_v1_12B_500K_300d_100q.tar'))

# nan vec for words missing in the model
nan_vec = np.zeros(navec['<unk>'].shape)
nan_vec[:] = np.nan

# parsing the order of stimulus presentations
pwi_trials = pd.DataFrame({})
for file in os.listdir(op.join('data', 'PWI_trial_lists')):
    with open(op.join('data', 'PWI_trial_lists', file), 'r', encoding='ANSI') as f:
        text = f.read().strip().split('\n')
        segm = [int(l.strip('=').split(' ')[0]) for l in text]
        patient = 'cprin' + file.split('.')[0]
        pwi_trials = pd.concat([pwi_trials, pd.DataFrame({'ID': [patient]*len(segm),
                                                          'Critical.segment': segm})])

# adding responses
pwi_trials = pwi_trials.merge(pwi_data[['ID', 'Critical.segment', 'Response']],
                              how = 'left', on = ['ID', 'Critical.segment'])
# adding response lemmas
pwi_trials = pwi_trials.merge(pwi_lemmas, how = 'left', on = ['Response'])

# checking nan values in lemmas
print('Checking nan values in lemmatization:')
print(pwi_trials.loc[pwi_trials['Lemma'].isna(), 'Response'].to_string())

# adding a column containing previous responses' lemmas
shifted_lemmas = []
for sub in pwi_trials['ID'].unique():
    subset = pwi_trials[pwi_trials['ID'] == sub]
    shifted_lemmas += list(subset['Lemma'].shift(1).values)
pwi_trials['prev_lemma'] = shifted_lemmas

# calculating similarity to previous lemma
dissimilarity = []
for ri, r in pwi_trials.iterrows():
    if r['Lemma'] != r['prev_lemma']:
        try:
            dissimilarity.append(1 - pearsonr(navec[r['Lemma']],
                                              navec[r['prev_lemma']]).statistic)
        except:
            dissimilarity.append(np.nan)
    else:
        dissimilarity.append(np.nan)
pwi_trials['consec_dissimilarity'] = dissimilarity

# calculating target-distractor similarity
pwi_stim = pwi_data[['Critical.segment', 'Condition', 'Target', 'Distractor']].drop_duplicates()

dissimilarity = []
for ri, r in pwi_stim.iterrows():
    if r['Target'] != r['Distractor']:
        try:
            dissimilarity.append(1 - pearsonr(navec[r['Target']],
                                              navec[r['Distractor']]).statistic)
        except:
            dissimilarity.append(np.nan)
    else:
        dissimilarity.append(np.nan)
        
pwi_stim['distr_dissimilarity'] = dissimilarity

# adding similarity values to the original df
pwi_data = pwi_data.merge(pwi_trials[['ID', 'Critical.segment', 'Lemma',
                                      'prev_lemma', 'consec_dissimilarity']],
                          how = 'left', on = ['ID', 'Critical.segment'])

pwi_data = pwi_data.merge(pwi_stim[['Critical.segment', 'distr_dissimilarity']],
                          how = 'left', on = 'Critical.segment')

# adding lemma frequency
pwi_data = pwi_data.merge(pwi_lemma_freq, how = 'left', on = 'Lemma')
pwi_data['logfreq'] = np.log(pwi_data['Frequency'])

print('Behavioral dataframe shape:', pwi_data.shape)
pwi_data.head()

Checking nan values in lemmatization:
0                                                     NaN
66                                                    NaN
132                                         не записалось
133                                            нет ответа
198                                         не записалось
264                                                   NaN
330                                                   NaN
396                                                   NaN
406                               спичка, конечно нет... 
416                                                   NaN
423                                            нет ответа
426                         за, за... ч.. к.. ну, не знаю
454                               за, за... ш... не надо 
458                ээ кр, нет... щас щас щас... ко... кок
462                                                   NaN
469                                            нет ответа
528                               

,Critical.segment,Target,Distractor,Condition,Response,ResponseType,RT,Comments,ID,Lemma,prev_lemma,consec_dissimilarity,distr_dissimilarity,Frequency,logfreq
0,1001,автобус,троллейбус,semantic,аптобус,1p,1072.0,-,cprin229,автобус,мотоцикл,0.439470,0.305089,64.8,4.171306
1,2001,автобус,автобус,congr,аптопульс,1p,1768.0,-,cprin229,автобус,лампа,0.906612,NaN,64.8,4.171306
2,3001,автобус,нож,unrel,NaN,99,NaN,не записалось,cprin229,NaN,None,NaN,0.900023,NaN,NaN
3,4001,акула,кит,semantic,акола,1p,1396.0,-,cprin229,акула,ведро,0.942681,0.618731,9.3,2.230014
4,5001,акула,акула,congr,акола,1p,1465.0,-,cprin229,акула,зажигалка,0.876260,NaN,9.3,2.230014


### Methods: Demographic and clinical summary

In [4]:
demo['Birth.date'] = pd.to_datetime(demo['Birth.date'], format = "%d.%m.%Y")
demo['Onset.date'] = pd.to_datetime(demo['Onset.date'], format = "%d.%m.%Y")
demo['MRI.date'] = pd.to_datetime(demo['MRI.date'], format = "%d.%m.%Y")
demo['Behaviour.date'] = pd.to_datetime(demo['Behaviour.date'], format = "%d.%m.%Y")

demo['age'] = demo.apply(lambda r: relativedelta(r['Behaviour.date'],
                                                 r['Birth.date']).years, axis = 1)
demo['months post stroke'] = demo.apply(lambda r: relativedelta(r['Behaviour.date'],
                                                                r['Onset.date']), axis = 1)
demo['months post stroke'] = demo['months post stroke'].apply(lambda x: x.years*12 + x.months)
demo['weeks to MRI'] = (demo['Behaviour.date'] - demo['MRI.date']).apply(lambda x: x.days/7)

Summary

In [5]:
print('Age descriptives')
print(demo['age'].describe())

print('\nTime post stroke, months')
print(demo['months post stroke'].describe())

print('\nTime between assessment and MRI, weeks')
print(np.abs(demo['weeks to MRI']).describe())

print('\nTime between assessment and MRI, weeks, for those with less than one year post stroke')
print(np.abs(demo.loc[demo['months post stroke']<12, 'weeks to MRI']).describe())

# saving demo for supplementay
demo[['ID', 'age', 'Sex', 'Handedness', 'Etiology',
      'Aphasia.type', 'Dysarhtria', 'months post stroke', 'weeks to MRI']].to_csv(op.join('output', 'suppl_1.csv'))

Age descriptives
count    25.000000
mean     55.520000
std      10.190355
min      37.000000
25%      47.000000
50%      56.000000
75%      66.000000
max      70.000000
Name: age, dtype: float64

Time post stroke, months
count    25.000000
mean     21.280000
std      24.247887
min       1.000000
25%       5.000000
50%      12.000000
75%      27.000000
max      88.000000
Name: months post stroke, dtype: float64

Time between assessment and MRI, weeks
count    25.000000
mean     16.754286
std      25.292221
min       0.428571
25%       1.142857
50%       2.428571
75%      26.000000
max      86.285714
Name: weeks to MRI, dtype: float64

Time between assessment and MRI, weeks, for those with less than one year post stroke
count    12.000000
mean      2.059524
std       1.211354
min       0.428571
25%       1.107143
50%       1.785714
75%       2.857143
max       4.285714
Name: weeks to MRI, dtype: float64


### Results

Summary of performance accuracy

In [6]:
print('all response types:', pwi_data['ResponseType'].unique())

pwi_data['correct'] = pwi_data['ResponseType'].str.endswith('1') + pwi_data['ResponseType'].str.endswith('1p')
pwi_data['incorrect'] = pwi_data['ResponseType'].isin(['3', '0', '32p', '2', '2p'])

print('all correct response types:', pwi_data.loc[pwi_data['correct'], 'ResponseType'].unique())
print('total n correct responses:', pwi_data['correct'].sum())
print('total n techical errors:', (pwi_data['ResponseType'] == '99').sum())

pwi_accuracy_summary = pwi_data[pwi_data['ResponseType'] != '99'].pivot_table(index='ID',
                                                                              values = ['correct', 'incorrect'],
                                                                              aggfunc = ['sum', 'mean'])
pwi_accuracy_summary.describe()

all response types: ['1p' '99' '1' '31p' '31' '21p' '3' '0' '32p' '2' '21' '2p']
all correct response types: ['1p' '1' '31p' '31' '21p' '21']
total n correct responses: 1476
total n techical errors: 39


sum                  mean           
         correct  incorrect    correct  incorrect
count  25.000000  25.000000  25.000000  25.000000
mean   59.040000   5.400000   0.916366   0.083634
std     5.630867   5.686241   0.087597   0.087597
min    45.000000   0.000000   0.692308   0.000000
25%    57.000000   1.000000   0.890625   0.016129
50%    61.000000   3.000000   0.953125   0.046875
75%    63.000000   7.000000   0.983871   0.109375
max    65.000000  20.000000   1.000000   0.307692

In [7]:
print('n incorrect responses:', pwi_data['incorrect'].sum())
print('% incorrect responses:', pwi_data['incorrect'].sum()*100/pwi_data.shape[0])

n incorrect responses: 135
% incorrect responses: 8.181818181818182


Cleaning data, summarizing the number of excluded trials

In [8]:
orig_n_trials = pwi_data.shape[0]
pwi_data_clean = pwi_data.copy()

# excluding incorrect responses
correct_filter = pwi_data_clean['ResponseType'].isin(['1', '1p'])
pwi_data_clean = pwi_data_clean.loc[correct_filter]
print('excluding incorrect trials:',
      sum(~correct_filter),
      round(sum(~correct_filter)*100/orig_n_trials, 1))

# excluding trials with missing RT
RT_filter = ~pwi_data_clean['RT'].isna()
pwi_data_clean = pwi_data_clean.loc[RT_filter]
print('excluding trials with missing RTs:',
      sum(~RT_filter),
      round(sum(~RT_filter)*100/orig_n_trials, 1))

# excluding trials with logRT > +- 3SDs
# centering logRT to individual participants' means
pwi_data_clean['logRT'] = np.log(pwi_data_clean['RT'])
RT_thresh = pwi_data_clean.pivot_table(index = ['ID', 'Condition'],
                                       values = 'logRT',
                                       aggfunc = ['mean', 'std']).reset_index()
RT_thresh['RT_thresh_lower'] = RT_thresh[('mean', 'logRT')] - 3*RT_thresh[('std', 'logRT')]
RT_thresh['RT_thresh_upper'] = RT_thresh[('mean', 'logRT')] + 3*RT_thresh[('std', 'logRT')]

pwi_data_clean = pwi_data_clean.merge(RT_thresh[['ID', 'Condition', 'RT_thresh_lower', 'RT_thresh_upper']].droplevel(1, axis=1),
                                      how = 'left',
                                      on = ['ID', 'Condition'])

RT_outlier_filter = (pwi_data_clean['logRT'] > pwi_data_clean['RT_thresh_lower']) * \
                    (pwi_data_clean['logRT'] < pwi_data_clean['RT_thresh_upper'])

pwi_data_clean = pwi_data_clean.loc[RT_outlier_filter]

print('excluding trials with outlier logRTs:',
      sum(~RT_outlier_filter),
      round(sum(~RT_outlier_filter)*100/orig_n_trials, 1))

print('clean dataframe shape:', pwi_data_clean.shape, '\npercentage of trials for modeling:', pwi_data_clean.shape[0]*100/pwi_data.shape[0])

pwi_nincl_summary = pwi_data_clean.pivot_table(index='ID',
                                               values = 'Critical.segment',
                                               aggfunc = 'count')
pwi_nincl_summary.describe()

excluding incorrect trials: 220 13.3
excluding trials with missing RTs: 160 9.7
excluding trials with outlier logRTs: 14 0.8
clean dataframe shape: (1256, 20) 
percentage of trials for modeling: 76.12121212121212


,Critical.segment
count,25.000000
mean,50.240000
std,9.713736
min,28.000000
25%,44.000000
50%,52.000000
75%,59.000000
max,62.000000


In [9]:
# adding neural predictors for model fitting
pwi_data_clean_withpreds = pwi_data_clean.copy()
pwi_data_clean_withpreds = pwi_data_clean_withpreds.merge(neural_predictors, how = 'left', on = 'ID')
print('Dataframe shape after adding neural predictors:', pwi_data_clean_withpreds.shape)
pwi_data_clean_withpreds.to_csv('pwi_data_clean_withpreds.csv', index=False)

Dataframe shape after adding neural predictors: (1256, 26)


T-test comparing target-distractor dissimilarity between conditions

In [10]:
ttest_ind(pwi_stim.loc[pwi_stim['Condition'] == 'semantic', 'distr_dissimilarity'],
          pwi_stim.loc[pwi_stim['Condition'] == 'unrel', 'distr_dissimilarity'],
          axis=0, equal_var=False, nan_policy='raise', alternative='two-sided')

TtestResult(statistic=-12.4240330273628, pvalue=2.5620986832115902e-15, df=40.03548502821155)

ANOVA testing whether response dissimilarity differs between conditions

In [11]:
f_oneway(pwi_data_clean.loc[pwi_data_clean['Condition'] == 'semantic', 'consec_dissimilarity'].dropna(), 
         pwi_data_clean.loc[pwi_data_clean['Condition'] == 'congr', 'consec_dissimilarity'].dropna(),
         pwi_data_clean.loc[pwi_data_clean['Condition'] == 'unrel', 'consec_dissimilarity'].dropna(),
         axis=0)

F_onewayResult(statistic=0.3831739661787659, pvalue=0.6817778823068653)

Correlating target-distractor dissimilarity and consecutive response dissimilarity

In [12]:
temp_corr_data = pwi_data_clean.dropna(subset=['consec_dissimilarity', 'distr_dissimilarity'])
pearsonr(temp_corr_data['consec_dissimilarity'], temp_corr_data['distr_dissimilarity'])

PearsonRResult(statistic=-0.030409832099655164, pvalue=0.40375053364077834)

Fitting mixed-effects models for behavioral effects (Supplementary Tables 2, 3)

In [13]:
cmd = 'C:\\"Program Files"\\R\\R-4.2.2\\bin\\Rscript utils\\behav_model_fitting.R'
output = subprocess.check_output(cmd, universal_newlines=True, shell=True)
print(output)

[1] "z-scoring continuous predictors:"
[1] "Age"                  "Post.onset.weeks"     "consec_dissimilarity"
[4] "distr_dissimilarity"  "logfreq"             
[1] "gender coding:"
  [,1]
f    1
m   -1
[1] "distractor condition coding"
         congr unrel
semantic     0     0
congr        1     0
unrel        0     1
Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: logRT ~ Condition * consec_dissimilarity + logfreq + Age + Sex +  
    Post.onset.weeks + (1 | ID) + (1 | Critical.segment)
   Data: pwi_data
Control: lmerControl(optimizer = "bobyqa")

REML criterion at convergence: 1187.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.5325 -0.6212 -0.1370  0.4316  5.6654 

Random effects:
 Groups           Name        Variance Std.Dev.
 Critical.segment (Intercept) 0.008094 0.08997 
 ID               (Intercept) 0.078206 0.27965 
 Residual                     0.136663 0.36968 
Number of obs: 1200, groups:  Critical.segment, 

Fitting mixed-effects models for lesion load effects (Supplementary Tables 4, 5)

In [14]:
cmd = 'C:\\"Program Files"\\R\\R-4.2.2\\bin\\Rscript utils\\neural_model_fitting.R'
output = subprocess.check_output(cmd, universal_newlines=True, shell=True)
print(output)

[1] "z-scoring continuous predictors:"
 [1] "Age"                  "Post.onset.weeks"     "consec_dissimilarity"
 [4] "distr_dissimilarity"  "logfreq"              "POper"               
 [7] "PTri"                 "PreCG"                "SLF_I"               
[10] "SLF_II"              
[1] "gender coding:"
  [,1]
f    1
m   -1
[1] "condition coding:"
         congr unrel
semantic     0     0
congr        1     0
unrel        0     1
[1] "variance inflation factors:"
                                 GVIF Df GVIF^(1/(2*Df))
POper                        2.299115  1        1.516283
Condition                    1.004116  2        1.001027
consec_dissimilarity         1.022653  1        1.011263
PTri                         2.708588  1        1.645779
SLF_I                        4.586733  1        2.141666
SLF_II                       4.512703  1        2.124312
PreCG                        4.021994  1        2.005491
logfreq                      1.018088  1        1.009004
Age           

Numerically assessing interference effects at the single-subject level

In [15]:
# calculcating individual effect sizes
from numpy import mean, std
from math import sqrt
def cohen_d(x,y):
    return (mean(x) - mean(y)) / sqrt((std(x, ddof=1) ** 2 + std(y, ddof=1) ** 2) / 2.0)

temp_id_vec = []
temp_consec_effect_vec = []
temp_lexint_effect_vec = []
temp_semint_effect_vec = []

for sub in pwi_data_clean['ID'].unique():

    subset = pwi_data_clean[pwi_data_clean['ID'] == sub]
    subset = subset.dropna(subset=['logRT', 'consec_dissimilarity'])
    
    temp_id_vec.append(sub)
    
    # calculating effect sizes
    temp_model = smf.ols(formula='logRT ~ consec_dissimilarity', data=subset).fit()
    temp_consec_effect_vec.append(temp_model.params['consec_dissimilarity'])
    temp_semint_effect_vec.append(cohen_d(subset.loc[subset['Condition'] == 'unrel', 'logRT'],
                                          subset.loc[subset['Condition'] == 'semantic', 'logRT']))
    temp_lexint_effect_vec.append(cohen_d(subset.loc[subset['Condition'] == 'congr', 'logRT'],
                                          subset.loc[subset['Condition'] == 'semantic', 'logRT']))


eff_size = pd.DataFrame({'ID': temp_id_vec,
                         'consec_eff': temp_consec_effect_vec,
                         'lexint_eff': temp_lexint_effect_vec,
                         'semint_eff': temp_semint_effect_vec})

# calculating N subjects showing interference effects
eff_size_binary = eff_size.copy()
eff_size_binary[['consec_eff', 'lexint_eff', 'semint_eff']] = eff_size_binary[['consec_eff', 'lexint_eff', 'semint_eff']]<0
print('N patients showing each interference effect:\n', eff_size_binary.sum()[['consec_eff', 'lexint_eff', 'semint_eff']])
print('Percent of patients showing each interference effect:\n', eff_size_binary.sum()[['consec_eff', 'lexint_eff', 'semint_eff']]*100/25)

N patients showing each interference effect:
 consec_eff    15
lexint_eff    22
semint_eff    12
dtype: object
Percent of patients showing each interference effect:
 consec_eff    60.0
lexint_eff    88.0
semint_eff    48.0
dtype: object


Figure 1

In [16]:
# filtering trials to plot task design
example_trials = pd.read_csv(op.join('data', 'example_trials_cprin325', 'example_trials.csv'))

# 1A - TASK DESIGN
cond_colors = {'Unrelated': '#de3d82', 'Congruent': '#9d57f4', 'Semantically\\nrelated': 'black'}

fig, ax = plt.subplots(nrows=6, ncols = 7,
                       figsize = (7.087, 3),
                       gridspec_kw=dict(height_ratios=[1.5,1,1.75,0.5,1,1],
                                        width_ratios=[1.5,1,1,1,1,1,1]))

for ir in range(6):
    for ic in range(7):
        if (ir == 4 and ic == 1) or (ir == 5 and ic == 1):
            ax[ir, ic].spines[['top', 'bottom', 'right']].set_visible(False)
            ax[ir, ic].set_xticks([])
        else:
            ax[ir, ic].set_axis_off()

ax[0,0].annotate('Stimulus features', (0, -0.4), xycoords='axes fraction',
                 fontsize = 7, weight='bold', va='bottom', ha = 'center', rotation=90)

ax[2,0].annotate('Response\nannotations', (0, 0.5), xycoords='axes fraction',
                 fontsize = 7, weight='bold', va='center', ha = 'center', rotation=90)

ax[4,0].annotate('Response latency\npredictors', (0, 0.5), xycoords='axes fraction',
                 fontsize = 7, weight='bold', va='center', ha = 'center', rotation=90)

# plotting row labels
ax[0,0].annotate('Stimulus', (0.15, 0.5), xycoords='axes fraction', fontsize = 7, va='center')
ax[1,0].annotate('Target', (0.15, 0.66), xycoords='axes fraction', fontsize = 7, va='center')
ax[1,0].annotate('Distractor', (0.15, 0.33), xycoords='axes fraction', fontsize = 7, va='center')
ax[2,0].annotate('Response lemma', (0.15, 0.7), xycoords='axes fraction', fontsize = 7, va='center')
ax[2,0].annotate('Latency, s', (0.15, 0.43), xycoords='axes fraction', fontsize = 7, va='center')
ax[2,0].annotate('Preceding response\nlemma', (0.15, 0.05), xycoords='axes fraction', fontsize = 7, va='center')
ax[3,0].annotate('Condition', (0.15, 0.5), xycoords='axes fraction', fontsize = 7, va='center')
ax[4,0].annotate('Target-distractor\ndissimilarity', (0.15, 0.5), xycoords='axes fraction', fontsize = 7, va='center')
ax[5,0].annotate('Sequential response\ndissimilarity', (0.15, 0.5), xycoords='axes fraction', fontsize = 7, va='center')

# plotting response parameters
for i, (ri, r) in enumerate(example_trials.iterrows()):
    ax[1,i+1].annotate(r['target_eng'], (0, 0.66), xycoords='axes fraction', fontsize = 7, va='center')
    ax[1,i+1].annotate(r['distractor_eng'], (0, 0.33), xycoords='axes fraction', fontsize = 7, va='center')
    
    sf, audio = wavfile.read(op.join('data', 'example_trials_cprin325', str(r['Critical.segment'])+'.wav'))
    tvec = np.linspace(0, audio.shape[0]/sf, audio.shape[0])
    ax[2,i+1].plot(tvec, (audio-audio.mean())/audio.max(), color = 'lightgrey', linewidth=0.5)
    ax[2,i+1].set_xlim(0, 2.2)
    ax[2,i+1].set_ylim(-1.2, 1)
    ax[2,i+1].hlines(y=-0.15, xmin=0, xmax = r['RT']/1000-0.05, color = 'black', linewidth=2.5)
    ax[2,i+1].annotate(round(r['RT']/1000, 3), (0, 0.3), xycoords='axes fraction', fontsize = 7, va='center')
    ax[2,i+1].annotate(r['Lemma_eng'], (0, 0.7), xycoords='axes fraction', fontsize = 7, va='center')
    ax[2,i+1].annotate(r['prev_lemma_eng'], (0, 0.05), xycoords='axes fraction', fontsize = 7, va='center')
    
    ax[3,i+1].annotate('\n'.join(r['Condition'].split('\\n')), (0, 0.5),
                       xycoords='axes fraction', fontsize = 7, va='center',
                       color = cond_colors[r['Condition']])
    if np.isnan(r['distr_dissimilarity']):
        ax[4,i+1].annotate('Not\ndefined', (0, 0.5), xycoords='axes fraction', fontsize = 7, va='center')
    ax[4,i+1].vlines(x=0.05, ymin=0, ymax = r['distr_dissimilarity'], color = 'black', linewidth=2.5)
    ax[4,i+1].set_ylim(0.3,1)
    ax[4,i+1].set_xlim(0,1)
    
    ax[5,i+1].vlines(x=0.05, ymin=0, ymax = r['consec_dissimilarity'], color = '#008c87', linewidth=2.5)
    ax[5,i+1].set_ylim(0.3,1)
    ax[5,i+1].set_xlim(0,1)
    
ax[4,1].set_yticks([0.3,1])
ax[5,1].set_yticks([0.3,1])
plt.tight_layout()
plt.subplots_adjust(hspace=0.3, wspace=0.1)

plt.savefig(op.join('output', 'figure1_top_v2.pdf'))

C:\Users\azyryanov\AppData\Local\Temp\ipykernel_8888\1511090745.py:45: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sf, audio = wavfile.read(op.join('data', 'example_trials_cprin325', str(r['Critical.segment'])+'.wav'))


In [17]:
# summarizing RTs for plotting
pwi_rt_summary = pwi_data_clean.copy().pivot_table(index = ['ID', 'Condition'],
                                                   values = 'logRT',
                                                   aggfunc = 'mean').reset_index()
pwi_rt_summary['meanRT'] = np.exp(pwi_rt_summary['logRT'])/1000


fig, ax = plt.subplot_mosaic([['semint_rt', 'lexint_rt', 'consec_rt', 'venn']],
                             figsize = (7.087, 2.2),
                             gridspec_kw=dict(width_ratios=[1.5,1.5,2.7,2]))

# some common styling
effect_color = {'semint_rt': '#de3d82', 'lexint_rt': '#9d57f4', 'consec_rt': '#008c87'}
sns.set_style('ticks')

def remove_half_violin(axis):
    xlim = axis.get_xlim()
    ylim = axis.get_ylim()
    for violin in axis.collections:
        bbox = violin.get_paths()[0].get_extents()
        x0, y0, width, height = bbox.bounds
        violin.set_clip_path(plt.Rectangle((x0-0.05, y0), (width / 2) + 0.05, height,
                                           transform=axis.transData))
    return axis

for a in ax:
    ax[a].spines[['top', 'right']].set_visible(False)

ax['semint_rt'].annotate('C', (-0.4, 1), xycoords='axes fraction', fontsize = 10, weight='bold', va='top')
ax['consec_rt'].annotate('D', (-0.25, 1), xycoords='axes fraction', fontsize = 10, weight='bold', va='top')
ax['venn'].annotate('E', (-0.4, 1), xycoords='axes fraction', fontsize = 10, weight='bold', va='top')

# PANEL C - DISTRACTOR INTERFERENCE EFFECTS
sns.violinplot(data=pwi_rt_summary[pwi_rt_summary['Condition'] != 'congr'],
               x='Condition', y='meanRT', hue=None,
               split=False, ax = ax['semint_rt'], cut = 0,
               dodge=False, inner=None, color='lightgrey',
               label = '_nolegend_')

sns.boxplot(data=pwi_rt_summary[pwi_rt_summary['Condition'] != 'congr'],
            x='Condition', y='meanRT', saturation=1, showfliers=False,
            width=0.2, boxprops={'zorder': 3, 'facecolor': 'none'}, ax=ax['semint_rt'])

for i, violin in enumerate(ax['semint_rt'].collections):
    bbox = violin.get_paths()[0].get_extents()
    x0, y0, width, height = bbox.bounds
    if i == 0:
        violin.set_clip_path(plt.Rectangle((x0-0.05, y0), (width / 2) + 0.05, height,
                                           transform=ax['semint_rt'].transData))
    else:
        violin.set_clip_path(plt.Rectangle((x0+width/2, y0), (width / 2) + 0.05, height,
                                           transform=ax['semint_rt'].transData))
        
for i, sub in enumerate(pwi_rt_summary['ID'].unique()):
    yvals = [pwi_rt_summary.loc[(pwi_rt_summary['Condition'] == c) & \
                                (pwi_rt_summary['ID'] == sub), 'meanRT'] for c in ['semantic', 'unrel']]
    if yvals[0].values[0]>yvals[1].values[0]:
        color = effect_color['semint_rt']
    else:
        color = 'black'
    ax['semint_rt'].plot([0.25,0.75], yvals, marker = 'o', markersize = 1,
                          linewidth = 0.5, color = color)

ax['semint_rt'].set_ylabel('Mean response latency, s')
ax['semint_rt'].set_yscale('log')
ax['semint_rt'].set_ylim(0.7,4.5)
ax['semint_rt'].set_yticks([0.7,1,2,3,4], labels=[0.7,1,2,3,4])
ax['semint_rt'].set_xticks([0,1], labels = ['Semantic\ncondition', 'Unrelated\ncondition'])
ax['semint_rt'].set_xlabel('')
ax['semint_rt'].spines[['bottom']].set_visible(False)
ax['semint_rt'].tick_params(axis='x', bottom=False)

sns.violinplot(data=pwi_rt_summary[pwi_rt_summary['Condition'] != 'unrel'],
               x='Condition', y='meanRT', hue=None,
               split=False, ax = ax['lexint_rt'], cut = 0,
               dodge=False, inner=None, color='lightgrey',
               label = '_nolegend_')

sns.boxplot(data=pwi_rt_summary[pwi_rt_summary['Condition'] != 'unrel'],
            x='Condition', y='meanRT', saturation=1, showfliers=False,
            width=0.2, boxprops={'zorder': 3, 'facecolor': 'none'}, ax=ax['lexint_rt'])

for i, violin in enumerate(ax['lexint_rt'].collections):
    bbox = violin.get_paths()[0].get_extents()
    x0, y0, width, height = bbox.bounds
    if i == 0:
        violin.set_clip_path(plt.Rectangle((x0-0.05, y0), (width / 2) + 0.05, height,
                                           transform=ax['lexint_rt'].transData))
    else:
        violin.set_clip_path(plt.Rectangle((x0+width/2, y0), (width / 2) + 0.05, height,
                                           transform=ax['lexint_rt'].transData))
        
for i, sub in enumerate(pwi_rt_summary['ID'].unique()):
    yvals = [pwi_rt_summary.loc[(pwi_rt_summary['Condition'] == c) & \
                                (pwi_rt_summary['ID'] == sub), 'meanRT'] for c in ['congr', 'semantic']]
    
    if yvals[0].values[0]<yvals[1].values[0]:
        color = effect_color['lexint_rt']
    else:
        color = 'black'
        
    ax['lexint_rt'].plot([0.25,0.75], yvals, marker = 'o', markersize = 1,
                          linewidth = 0.5, color = color)

ax['lexint_rt'].set_ylabel(None)
ax['lexint_rt'].set_yscale('log')
ax['lexint_rt'].set_ylim(0.7,4.5)
ax['lexint_rt'].set_yticks([0.7,1,2,3,4], labels=[])
ax['lexint_rt'].set_xticks([0,1], labels = ['Congruent\ncondition', 'Semantic\ncondition'])
ax['lexint_rt'].set_xlabel('')
ax['lexint_rt'].spines[['bottom']].set_visible(False)
ax['lexint_rt'].tick_params(axis='x', bottom=False)
ax['lexint_rt'].set_xlim(ax['lexint_rt'].get_xlim()[::-1])

# plotting sequential interference effects on RT
for i, sub in enumerate(pwi_data_clean['ID'].unique()):
    
    subset_consec = pwi_data_clean.loc[(pwi_data_clean['ID'] == sub), ['logRT', 'consec_dissimilarity']]
    subset_consec = subset_consec.dropna()
   
    # plotting
    ax['consec_rt'].scatter(subset_consec['consec_dissimilarity'], subset_consec['logRT'],
                            s=0.8, color = 'lightgrey', alpha = 0.5)
    
    temp_effsize = eff_size.loc[eff_size['ID'] == sub, 'consec_eff'].values[0]
    
    if temp_effsize<0:
        color = effect_color['consec_rt']
    else:
        color = 'black'
        
    sns.regplot(data = subset_consec,
                x = subset_consec['consec_dissimilarity'],
                y = subset_consec['logRT'],
                ax = ax['consec_rt'],
                ci = None, color = color, marker = '',
                line_kws={'linewidth':0.5, 'alpha':1})

ax['consec_rt'].set_ylabel('Response latency, s')
ax['consec_rt'].set_xlabel('Sequential response dissimilarity')
ax['consec_rt'].spines[['top', 'right']].set_visible(False)
ax['consec_rt'].set_yticks(np.log([500,1000,3000,10000]), labels=[0.5,1,3,10])

# PANEL D -- VENN DIAGRAM
ax['venn'].set_axis_off()
lexint_ids = set(eff_size_binary.loc[eff_size_binary['lexint_eff'], 'ID'])
semint_ids = set(eff_size_binary.loc[eff_size_binary['semint_eff'], 'ID'])
consec_ids = set(eff_size_binary.loc[eff_size_binary['consec_eff'], 'ID'])

v = venn3([lexint_ids, semint_ids, consec_ids], ('', '', ''), ax = ax['venn'])

v.patches[0].set_color('#9d57f4')
v.patches[2].set_color('#de3d82')
v.patches[3].set_color('#008c87')
v.patches[6].set_color('black')

plt.tight_layout()
plt.subplots_adjust(hspace=0.3, wspace=0.5)
plt.savefig(op.join('output', 'figure1_bottom_v2.pdf'))

The relation between interference effect sizes

In [18]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
effect_size_model = smf.ols(formula='lexint_eff ~ consec_eff + semint_eff',
                            data=eff_size).fit()
print(effect_size_model.summary())

                            OLS Regression Results                            
Dep. Variable:             lexint_eff   R-squared:                       0.090
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     1.093
Date:                Sat, 27 Jul 2024   Prob (F-statistic):              0.353
Time:                        16:47:24   Log-Likelihood:                -25.591
No. Observations:                  25   AIC:                             57.18
Df Residuals:                      22   BIC:                             60.84
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.7068      0.146     -4.842      0.0

Summarizing lesion volumes

In [19]:
# lesion volume summary
# calculating normalized lesion volumes
lesion_vols = cortical_loads.groupby('ID')[['numVoxNotZero']].sum()/1000
lesion_vols = lesion_vols.rename({'numVoxNotZero': 'lesion_volume'}, axis=1)

lesion_vols.describe()

,lesion_volume
count,25.000000
mean,65.475440
std,60.088772
min,5.532000
25%,27.858000
50%,44.077000
75%,85.740000
max,213.998000


Figure 2. Lesion characteristics

* A -- lesion overlay (plotted separately with MRICroGL)
* B -- lesion load distributions
* C -- lesion load corrplot

In [20]:
fig, ax = plt.subplot_mosaic([['overlay', 'load_distr', 'load_corrmat']],
                             figsize = (7.087,2.5),
                             gridspec_kw=dict(width_ratios=[1.5,1.3,1]))

# plotting lesion load distributions
inf_rois = ['PreCG', 'POper', 'PTri']
inf_colors = ['#ffbb63', '#ffb2ce', '#dbbbfe']
sup_rois = ['SLF_I', 'SLF_II', 'SLF_III']
sup_colors = ['#65dad2', '#67dea8', '#bce92a']

neural_predictors_long = pd.melt(neural_predictors,
                                 id_vars='ID',
                                 value_vars=inf_rois + sup_rois,
                                 var_name='roi',
                                 value_name='load')

sns.violinplot(data=neural_predictors_long,
               x='roi', y='load', hue='roi',
               dodge=False, inner=None, color = 'grey', order = inf_rois + sup_rois,
               palette = inf_colors + sup_colors,
               ax = ax['load_distr'], cut=0, linewidth=0)

for i, sub in enumerate(neural_predictors['ID'].unique()):

    ax['load_distr'].plot(np.arange(len(inf_rois)),
                          neural_predictors.loc[neural_predictors['ID'] == sub, inf_rois].values.squeeze(),
                          marker = 'o', markersize = 1,
                          linewidth = 0.3, color = 'black')
    
    ax['load_distr'].plot(np.arange(len(sup_rois)) + 3,
                          neural_predictors.loc[neural_predictors['ID'] == sub, sup_rois].values.squeeze(),
                          marker = 'o', markersize = 1,
                          linewidth = 0.3, color = 'black')

labels = [l.replace('_', ' ') for l in inf_rois + sup_rois]
ax['load_distr'].set_xticks(np.arange(len(inf_rois + sup_rois)), labels = labels)
ax['load_distr'].set_ylabel('Lesion load')
ax['load_distr'].set_xlabel('Region of interest')

# add inferior frontal and fronto-parietal labels
ax['load_distr'].set_ylim(0.0,1)
ax['load_distr'].set_yticks([0,0.5,1])
ax['load_distr'].text(1,1,'Inferior frontal\ncortex', ha='center', va='top')
ax['load_distr'].text(4,1,'Fronto-parietal\nconnectivity', ha='center', va='top')

# remove spines
ax['load_distr'].spines[['top', 'bottom', 'right']].set_visible(False)
ax['load_distr'].tick_params(axis='x', bottom=False)

corr = neural_predictors[inf_rois + sup_rois].corr()
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
np.fill_diagonal(mask, False)

heatmap = sns.heatmap(corr,
                      annot=True,
                      fmt='.2f',
                      linewidths=0.5,
                      cmap='RdBu_r',
                      mask=mask,
                      ax=ax['load_corrmat'],
                      square=True,
                      cbar=False, vmin=-1,vmax=1)

# add colorbar
norm = plt.Normalize(-1,1)
sm = plt.cm.ScalarMappable(cmap="RdBu_r", norm=norm)

cax = ax['load_corrmat'].inset_axes([0.6, 0.95, 0.35, 0.05], in_layout = False)
cb = fig.colorbar(sm, cax=cax, orientation = 'horizontal')
cax.set_xticks([-1,0,1])
cax.set_xlabel('Pearson $r$')

ax['load_corrmat'].tick_params(axis='both', size=0)

ax['overlay'].set_xticks([])
ax['overlay'].set_yticks([])

norm = plt.Normalize(0,19)
sm = plt.cm.ScalarMappable(cmap="plasma", norm=norm)
cax = ax['overlay'].inset_axes([0, 0.95, 0.3, 0.04], in_layout = False)
cb = fig.colorbar(sm, cax=cax, orientation = 'horizontal')
cax.set_xticks([0,10,19])
cax.set_xlabel('N lesioned')

# overlay code Z -62 -58 -50; -44 -38 -32; -26 -20 -14

plt.tight_layout()
plt.savefig(op.join('output', 'figure2_v2.pdf'))

Figure 3

In [21]:
# preparing condition averages for plotting

# getting quantiles for sequential resp dissimilarity by subject
def q25(vals):
    return vals.quantile(.25)

def q75(vals):
    return vals.quantile(.75)

consec_dissim_quantiles = pwi_data_clean.pivot_table(index = 'ID',
                                                     values = 'consec_dissimilarity',
                                                     aggfunc = [q25, q75])
consec_dissim_quantiles.columns = consec_dissim_quantiles.columns.to_series().str.join('_')

# binning sequential resp dissimilarity in the orig DF
pwi_data_clean = pwi_data_clean.merge(consec_dissim_quantiles, how = 'left', on ='ID')
pwi_data_clean['consec_bin'] = 'mid'
pwi_data_clean.loc[pwi_data_clean['consec_dissimilarity']<=pwi_data_clean['q25_consec_dissimilarity'],
                   'consec_bin'] = 'low'
pwi_data_clean.loc[pwi_data_clean['consec_dissimilarity']>=pwi_data_clean['q75_consec_dissimilarity'],
                   'consec_bin'] = 'high'

# averaging within condition/bin
mean_distrcond = pwi_data_clean.pivot_table(index = ['ID', 'Condition'],
                                            values = 'logRT',
                                            aggfunc = 'mean').reset_index()
mean_distrcond = mean_distrcond.pivot(index='ID', columns='Condition', values='logRT')

mean_consecbin = pwi_data_clean[pwi_data_clean['Condition'] != 'congr'].pivot_table(index = ['ID', 'consec_bin'],
                                                                                    values = 'logRT',
                                                                                    aggfunc = 'mean').reset_index()
mean_consecbin = mean_consecbin.pivot(index='ID', columns='consec_bin', values='logRT')

means_toplot = mean_distrcond.merge(mean_consecbin, how = 'left', on = 'ID')
means_toplot = means_toplot.merge(neural_predictors, how = 'left', on = 'ID').set_index('ID')
means_toplot.head()

,congr,semantic,unrel,high,low,mid,POper,PTri,PreCG,SLF_I,SLF_II,SLF_III
ID,,,,,,,,,,,,
cprin229,7.239626,7.143659,7.189506,7.226892,7.203358,7.102963,0.114805,0.000000,0.124404,0.002511,0.148373,0.230125
cprin268,6.978686,7.187864,7.173845,7.268680,7.009238,7.214519,0.420899,0.027185,0.122674,0.000093,0.094437,0.266915
cprin282,7.198122,7.688929,7.660220,7.744902,7.718803,7.609114,0.008798,0.010055,0.004497,0.001285,0.049046,0.087303
cprin293,7.309598,7.657168,7.628401,7.537637,7.244165,7.825685,0.371566,0.331276,0.262587,0.243034,0.895784,0.795703
cprin323,7.326393,7.553640,7.604545,7.640400,7.285595,7.757715,0.356777,0.009789,0.103334,0.000000,0.090493,0.259061


In [22]:
# taking interaction stats from R models 
sig_inter = {'POper_high_low': True,
             'POper_congr_semantic': True,
             'POper_unrel_semantic': False,
             'PTri_high_low': True,
             'PTri_congr_semantic': True,
             'PTri_unrel_semantic': False,
             'SLF_I_high_low': False,
             'SLF_I_congr_semantic': True,
             'SLF_I_unrel_semantic': False}

fig, ax = plt.subplots(nrows = 3, ncols = 4,
                       figsize = (7.087,4.75),
                       gridspec_kw=dict(width_ratios=[0.9,1,1,0.2]),
                       sharey=True, sharex=True)

effect_color = {'unrel_semantic': '#de3d82', 'congr_semantic': '#9d57f4', 'high_low': '#008c87'}

for r, roi in enumerate(['PTri', 'POper', 'SLF_I']):
    for c, effect in enumerate(['unrel_semantic', 'congr_semantic', 'high_low']):
        
        if sig_inter[f'{roi}_{effect}']:
            color = effect_color[effect]
            alpha = 1
        else:
            color = 'lightgrey'
            alpha = 0.5
        
        # plotting single-subject interference effects (vertical lines)
        condpair = effect.split('_')
        for s in pwi_data_clean['ID'].unique():
            load = means_toplot.loc[s, roi]
            yvals = means_toplot.loc[s, condpair].values
            
            ax[r,c].plot(np.zeros(2) + load,
                         yvals,
                         color = 'black',
                         alpha = alpha,
                         linewidth = 0.8)
        
        # plotting regression fits within condition (thick lines)
        for pi, p in enumerate(condpair):
            if pi == 1:
                color = 'black'
            ax[r,c].scatter(means_toplot[roi], means_toplot[p], color = color, s = 2)
            sns.regplot(data=means_toplot, x=roi, y=p, ax=ax[r,c], ci=None,
                        marker ='', line_kws={'linewidth':1.5, 'alpha':alpha}, color = color)
        
        ax[r,c].set_ylabel('')
        ax[r,c].set_xlabel('')
        
# setting x and y ranges/ticks
for r in range(3):
    for c in range(3):
        ax[r,c].set_xlim(-0.025, 0.5)
        ax[r,c].set_xticks([0,0.25,0.5])
        ax[r,c].spines[['top', 'right']].set_visible(False)

for r in range(3):
    ax[r,0].set_yticks(np.log([1000,2000,3000,4000]), labels = [1,2,3,4])

ax[2,0].set_xlabel('Lesion load')
ax[2,0].set_ylabel('Response latency, s')


    
for c, effect in enumerate(['Distractor interference:\nsemantically related vs. unrelated',
                            'Distractor interference:\nsemantically related vs. congruent']): 
    temp_annot = ax[0,c].annotate(effect, (0, 1.05), xycoords='axes fraction', fontsize = 7,
                                  ha = 'left')

temp_annot = ax[0,2].annotate('Sequential response interference\nlow vs. high sequential dissimilarity',
                              (0, 1.05), xycoords='axes fraction',
                              fontsize = 7, ha = 'left')

# adding ROI and effect annotations
for r, (roi, fullroi) in enumerate([('PTri', 'Pars\ntriangularis'),
                                    ('POper', 'Pars\nopercularis'),
                                    ('SLF_I', 'Superior longitudinal\nfasciculus I')]): 
    temp_annot = ax[r,3].annotate(fullroi, (0.5, 0.2), xycoords='axes fraction',
                                  va = 'center', ha = 'center', fontsize = 7, color = 'black')
    ax[r,3].axis("off")
    
plt.tight_layout()
plt.savefig(op.join('output', 'figure3_v3.pdf'))